In [ ]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web

# 信号前沿分析
pandas pandas_datareader.data  pdr.data.get_data_yahoo()  
pandas dataframe d.asfreq()  
pandas dataframe d.fillna()  
pandas dataframe d.head()  

In [ ]:
names  = ['AAPL','GOOG','MSFT','GS','MS','BAC','C']
def get_px(stock,start,end):
    df =web.get_data_yahoo(stock,start,end)
    return df['Adj Close']

px = pd.DataFrame({n: get_px(stock=n,start='2009/1/1',end='2012/6/1') for n in names})
px = px.asfreq(freq='B').fillna(method='pad')
px.head() 

In [ ]:
'''
绘制每只股票的累计收益
pandas dataframe d.pct_change()
pandas dataframe d.add()
pandas dataframe d.cumprod()  
pandas dataframe d.sub()
pandas dataframe d.plot.line()
'''
rets = px.pct_change()
rets.add(1).cumprod().sub(1).plot.line()